Import necessary modules.

In [1]:
import random
import math
import simpy
import json
from sys import argv
import statistics

Calculate N. Last three digits of our student numbers are 210, 024 and 123.

In [2]:
def get_N():
    s = 210 + 24 + 123
    if s > 1000:
        return s
    elif s > 10:
        return s + 1000
    else:
        return s * 300


N = get_N()

K, Lambda, Mu1 and Mu2 are already given.

In [3]:
K = math.ceil(N / 24)

Lambda = 1 / 300

Mu1 = 1 / 6

Mu2 = 1 / 10

Function for calculating Mu3 as specified in the description.

Mu3 will be set with that function before running the simulation.

In [4]:
def get_Mu3():
    r = random.uniform(1, 2)
    return 1 / (1 / Mu1 * r)


Mu3 = None  # we will set it before running the simulation

Event class. It represents events of the simulation.

In [5]:
class Event:
    def __init__(self, event_time, event_type, patient_index):
        self.event_time = event_time
        self.event_type = event_type
        self.patient_index = patient_index
        self.l = hospital.count + homes.count
        self.l_hosp = hospital.count

    def get_json(self):
        return {
            "event_time": self.event_time,
            "event_type": self.event_type,
            "patient_index": self.patient_index,
            "sick_people_count": self.l,
            "hospital_count": self.l_hosp
        }

Define the variables that we are going to need.

In [6]:
Events = []              # list of events
L = []                   # number of sick people at time t
L_hosp = []              # number of people in the hospital at time t
L_home = []              # number of people in their homes at time t
Sickness_times = []      # a list of all sickness times
project_json = []        # this will hold all the information of simulations

env = None
hospital = None; homes = None
L_monitor_process = None; L_hosp_monitor_process = None; L_home_monitor_process = None

 ### The class definition for the patients arriving at the modeled system. 
 - Constructor: `__init__` function: When they are created, they immediatelly initiate a call (i.e. activate the call process) with `get_sick` function, `get_sick` function handles the sickness, waits for the next sickness, then calls itself
     - `is_filler_iteration` means the initial run of get_sick function will emplace patient to the hospital immediately (for K/2 or K patients to fill hospital at t=0)
     - `is_initial_iteration` means the initial run of get_sick function will just wait for the next sickness (for initial creation of the patients).
 - `get_sick` function: 
    - A patient stays at home with 80% probability. His/her healing time is exponentially distributed with Mu2.
    - A patient goes to hospital with 20% probability.
       - If s/he is accepted to hospital, his/her healing time is exponentially distributed with Mu1.
       - If s/he is not accepted to hospital due to full capacity, his/her healing time is exponentially distributed with Mu3.

In [7]:
class Patient:
    def __init__(self, index, is_filler_iteration=False, is_first_iteration=False):
        self.index = index
        if is_filler_iteration:
            self.action = env.process(self.get_sick(is_filler_iteration=True))
        elif is_first_iteration:
            self.action = env.process(self.get_sick(is_first_iteration=True))
        else:
            Events.append(Event(env.now, "arrival", self.index))
            self.action = env.process(self.get_sick())

    def get_sick(self, is_filler_iteration=False, is_first_iteration=False):
        if not is_first_iteration:
            if is_filler_iteration:
                with hospital.request() as req:
                    yield req
                    sickness_time = random.expovariate(Mu1)
                    yield env.timeout(sickness_time)
            else:
                rand = random.random()
                if rand < 0.8:
                    with homes.request() as req:
                        yield req
                        sickness_time = random.expovariate(Mu2)
                        yield env.timeout(sickness_time)
                else:
                    req = hospital.request()
                    results = yield req | env.timeout(0)
                    if req in results:
                        sickness_time = random.expovariate(Mu1)
                        yield env.timeout(sickness_time)
                        hospital.release(req)
                    else:
                        hospital.release(req)
                        with homes.request() as req2:
                            yield req2
                            sickness_time = random.expovariate(Mu3)
                            yield env.timeout(sickness_time)
            Events.append(Event(env.now, "departure", self.index))
            Sickness_times.append(sickness_time)

        inter_arrival = random.expovariate(Lambda)
        yield env.timeout(inter_arrival)
        Patient(self.index)

`N` patients are generated with exponential distribution. initial_patient_count determines the number of patients in hospital at the beginning.

In [8]:
def patient_generator(initial_patient_count=0):
    for i in range(1, initial_patient_count+1):
        Patient(i, is_filler_iteration=True)
    for i in range(initial_patient_count+1, N+1):
        Patient(i, is_first_iteration=True)
        yield env.timeout(0)

Defining monitors. They will keep track of number of patient in the system, in the hospital, and in the homes, respectively

In [9]:
def L_monitor():
    while True:
        L.append(hospital.count + homes.count)
        yield env.timeout(1)

In [10]:
def L_hosp_monitor():
    while True:
        L_hosp.append(hospital.count)
        yield env.timeout(1)

In [11]:
def L_home_monitor():
    while True:
        L_home.append(homes.count)
        yield env.timeout(1)

Functions for running and resetting the simulation

In [12]:
def reset_variables():
    global Events, L, L_hosp, L_home, Sickness_times

    Events = []
    L = []
    L_hosp = []
    L_home = []
    Sickness_times = []


def run_simulation(initial_patient_count=0, time_limit=None, event_limit=None):
    global env, hospital, homes, L_monitor_process, L_hosp_monitor_process, L_home_monitor_process

    env = simpy.Environment()
    hospital = simpy.Resource(env, capacity=K)  # Hospital has K capacity.
    homes = simpy.Resource(env, capacity=int(1e100))  # Homes has infinite capacity.
    env.process(patient_generator(initial_patient_count))
    L_monitor_process = env.process(L_monitor())
    L_hosp_monitor_process = env.process(L_hosp_monitor())
    L_home_monitor_process = env.process(L_home_monitor())
    if time_limit is not None:
        env.run(until=time_limit)
    if event_limit is not None:
        while len(Events) < event_limit:
            env.step()

Helper functions for formatting the simulation data as a Python dictionary

In [13]:
def get_events_json():
    return list(map(lambda e: e.get_json(), Events))


def get_simulation_json():
    events = get_events_json()
    return {
        "events":events,
        "long_run_probability_of_the_hospital_being_empty": L_hosp[1:].count(0) / (len(L_hosp) - 1) if len(L_hosp) != 0 else None,
        "long_run_probability_of_the_hospital_being_full": L_hosp.count(K) / len(L_hosp) if len(L_hosp) != 0 else None,
        "average_number_of_occupied_beds_in_the_hospital": sum(L_hosp[1:]) / len(L_hosp[1:]) if len(L_hosp) != 0 else None,
        "average_number_of_sick_people_in_the_population": sum(L[1:]) / len(L[1:]) if len(L) != 0 else None,
        "average_proportion_of_sick_people_in_the_population": (sum(L)-L[0]) / (len(L)-1) / N if len(L) != 0 else None,
        "total_average_sickness_time": sum(Sickness_times) / len(Sickness_times) if len(Sickness_times) != 0 else None
    }


Running the simulation for 50 events

In [14]:
# for initial_patient_count in (0, math.floor(K/2), K):
#     random.seed(1234)
#     Mu3 = get_Mu3()
#     reset_variables()
#     run_simulation(initial_patient_count=initial_patient_count,event_limit=50)
#     sim_json = get_simulation_json()
#     events = sim_json['events']
#     sim_json.pop('events')
#     sim_json.update({'simulation time': events[len(events)-1]['event_time']})    
#     project_json.append({
#         "event_limit": 50,
#         "initial_patient_count": initial_patient_count,
#         "time_limit": None,
#         "seed": 1234,
#         "results": sim_json
#     })

Running the simulation for combinations of three different inital patient counts, three different time limits, and three different seeds.

In [15]:
for initial_patient_count in (0, math.floor(K/2), K):
    for time_limit in (1000,10000,100000):
        for seed in (123, 456, 789):
            random.seed(seed)
            Mu3 = get_Mu3()
            reset_variables()
            run_simulation(initial_patient_count=initial_patient_count, time_limit=time_limit)
            sim_json = get_simulation_json()
            sim_json.pop('events')
            project_json.append({
                "event_limit": None,
                "initial_patient_count": initial_patient_count,
                "time_limit": time_limit,
                "seed": seed,
                "results": sim_json
            })

Finally, we are writing the results to the file.

In [16]:
json.dump(project_json, open("./output.txt", "w+"), indent=4)

Calculating mean and variances of the results.

In [31]:
lst = json.load(open(r"./all_time_without_events.txt", "r"))
lrpohbe = []
lrpohbf = []
anoobith = []
anospitp = []
apospitp = []
tast = []
for ev in lst:
    res = ev["results"]
    lrpohbe.append(res["long_run_probability_of_the_hospital_being_empty"])
    lrpohbf.append(res["long_run_probability_of_the_hospital_being_full"])
    anoobith.append(res["average_number_of_occupied_beds_in_the_hospital"])
    anospitp.append(res["average_number_of_sick_people_in_the_population"])
    apospitp.append(res["average_proportion_of_sick_people_in_the_population"])
    tast.append(res["total_average_sickness_time"])
output_json = {
"Means": {
"long_run_probability_of_the_hospital_being_empty": statistics.mean(lrpohbe),
"long_run_probability_of_the_hospital_being_full": statistics.mean(lrpohbf),
"average_number_of_occupied_beds_in_the_hospital": statistics.mean(anoobith),
    "average_number_of_occupied_beds_in_the_hospital_initially_empty": statistics.mean([x['results']['average_number_of_occupied_beds_in_the_hospital'] for x in lst[:9]]),
    "average_number_of_occupied_beds_in_the_hospital_initially_full": statistics.mean([x['results']['average_number_of_occupied_beds_in_the_hospital'] for x in lst[-9:]]),
"average_number_of_sick_people_in_the_population": statistics.mean(anospitp),
    "average_number_of_sick_people_in_the_population_initially_empty": statistics.mean([x['results']['average_number_of_sick_people_in_the_population'] for x in lst[:9]]),
    "average_number_of_sick_people_in_the_population_initially_full": statistics.mean([x['results']['average_number_of_sick_people_in_the_population'] for x in lst[-9:]]),
"average_proportion_of_sick_people_in_the_population": statistics.mean(apospitp),
"total_average_sickness_time": statistics.mean(tast),    
    "total_average_sickness_time_initially_empty": statistics.mean([x['results']['total_average_sickness_time'] for x in lst[:9]]),
    "total_average_sickness_time_initially_full": statistics.mean([x['results']['total_average_sickness_time'] for x in lst[-9:]]),   
},
"Variances": {
"long_run_probability_of_the_hospital_being_empty": statistics.variance(lrpohbe),
"long_run_probability_of_the_hospital_being_full": statistics.variance(lrpohbf),
"average_number_of_occupied_beds_in_the_hospital": statistics.variance(anoobith),
"average_number_of_sick_people_in_the_population": statistics.variance(anospitp),
"average_proportion_of_sick_people_in_the_population": statistics.variance(apospitp),
"total_average_sickness_time": statistics.variance(tast)
}}
json.dump(output_json, open("./statistics.txt", "w+"), indent=4)

### Now experiments are done. Let's move on to theory.

 Calculating theoretical probability of hospital being empty.

In [18]:
P0 = 1/sum([math.comb(N,n) * math.pow(0.2*Lambda/Mu1,n) for n in range(0,K)])

Calculating theoretical probabilities of number of people in hospital.
`Pn[k]` represents probability of k people in hospital.

In [19]:
Pn = [P0] +[math.factorial(N) / math.factorial(N-k) / math.factorial(k) * math.pow(0.2*Lambda/Mu1,k) * P0 for k in range(1,K+1)]

Calculating necessary theoretical values using machine repair formulation.

Write theoretical values to the file.

In [20]:
avg_number_of_people_in_the_hosp = sum([n*Pn[n] for n in range(0,K)])
hospital_arrival_rate = 0.2 * Lambda * N
hospital_effective_arrival_rate = 0.2 * Lambda * (N - avg_number_of_people_in_the_hosp)
avg_sickness_time_in_hospital = avg_number_of_people_in_the_hosp / hospital_effective_arrival_rate
avg_sickness_time = 0.2  / Mu1 * hospital_effective_arrival_rate / hospital_arrival_rate + 0.8  / Mu2 + 0.2  / Mu3 * (1 - hospital_effective_arrival_rate / hospital_arrival_rate)

theoretical_values_json = {
    "probability_of_the_hospital_being_empty" : P0,
    "probability_of_the_hospital_being_full" : Pn[K],
    "average_number_of_occupied_beds_in_the_hospital" : avg_number_of_people_in_the_hosp,
    "total_average_sickness_time" : avg_sickness_time,
    "hospital_effective_arrival_rate / hospital_arrival_rate" : hospital_effective_arrival_rate / hospital_arrival_rate,
}

In [27]:
json.dump(theoretical_values_json, open("./theoretical_output.txt", "w+"), indent=4)